In [1]:
# Get today date

# Extract hourly data from today and yesterday

# Convert hourly data into a dataframe

# Preprocess the dataframe
    # Differencing
    
    # Normalization (Use training_mean and training_std)

# Convert the dataset into a Window (Input width = 48)

# Load the AR model

# Predict future output




In [2]:
# Dates
# import datetime as dt
from datetime import datetime, timedelta
import pytz

# REE API request
import requests
import json

# Convert extracted data into dataframe
import pandas as pd

# WindowGenerator
from window_generator import WindowGenerator

# Extract data in JSON format

In [3]:
def extract_json_data(day):
    URLprices = "https://apidatos.ree.es/es/datos/mercados/precios-mercados-tiempo-real"
    URLdemand = "https://apidatos.ree.es/es/datos/demanda/demanda-tiempo-real"
    URLEmissions = "https://apidatos.ree.es/es/datos/generacion/evolucion-estructura-generacion-emisiones-asociadas"
    
    startHour = "00:00"
    endHour = "23:59"
    
    date_start = str(day.year) + "-" + str(day.month) + "-" + str(day.day) + "T" + startHour
    date_end = str(day.year) + "-" + str(day.month) + "-" + str(day.day) + "T" + endHour
    
    PARAMS = {'start_date':date_start, 'end_date':date_end,"time_trunc":"hour", 'geo_limit': 'peninsular'}
    
    prices = requests.get(url=URLprices, params=PARAMS)
    demands = requests.get(url=URLdemand, params=PARAMS)
    emissions = requests.get(url=URLEmissions, params=PARAMS)
    
    pricesJ = json.dumps(prices.json(), indent=4)
    demandJ = json.dumps(demands.json(), indent=4)
    emissionsJ = json.dumps(emissions.json(), indent=4)
    
    return pricesJ, demandJ, emissionsJ
    

In [4]:
timezone = pytz.timezone('Europe/Madrid')

today = datetime.now(tz=timezone)
yesterday = today - timedelta(days=1)
day_zero = yesterday - timedelta(days=1)


In [5]:
pricesJ_today, demandJ_today, emissionsJ_today = extract_json_data(today)
pricesJ_yesterday, demandJ_yesterday, emissionsJ_yesterday = extract_json_data(yesterday)
pricesJ_day_zero, demandJ_day_zero, emissionsJ_day_zero = extract_json_data(day_zero)

# Convert json data into a dataframe

In [6]:
type(pricesJ_day_zero)

str

In [7]:
def clean_datetime(timestamp):
    r = timestamp.replace(".000+01:00", "")
    r = r.replace(".000+02:00", "")
    return r

In [8]:
def convert_json_into_dataframe(price_json, demand_json, emissions_json, today = False):
    
    df_price = pd.DataFrame(columns=['timestamp', 'price'])
    df_demand = pd.DataFrame(columns=['timestamp', 'demand'])
    df_emissions = pd.DataFrame(columns=['timestamp', 'emissions'])
    
    # Generate price dataframe
    price_json = json.loads(price_json)
    prices_values = price_json['included'][0]['attributes']['values']
    for price_value in prices_values:
        row = {'timestamp': price_value['datetime'],
                   'price': price_value['value']}
        df_price = df_price.append(row, ignore_index=True)
        
    # Generate demand dataframe
    demand_json = json.loads(demand_json)
    if(today):
        demand_values = demand_json['included'][1]['attributes']['values']
    else:
        demand_values = demand_json['included'][0]['attributes']['values']

    for demand_value in demand_values:
        row = {'timestamp': demand_value['datetime'],
                   'demand': demand_value['value']}
        df_demand = df_demand.append(row, ignore_index=True)
    
    # Generate emissions dataframe
    emissions_json = json.loads(emissions_json)
    emissions_values = emissions_json['included'][0]['attributes']['values']

    for emissions_value in emissions_values:
        row = {'timestamp': emissions_value['datetime'],
                'emissions': emissions_value['value']}
        df_emissions = df_emissions.append(row, ignore_index=True)    
    
    df_merged = df_price.merge(df_demand, on='timestamp', how='inner')
    
    df_emissions['timestamp'] = df_price['timestamp']
    df_merged = df_merged.merge(df_emissions, on='timestamp', how='inner')
    
    df_merged['timestamp'] = df_merged['timestamp'].apply(clean_datetime)
    
    # Separate two columns by " "
    df_merged[['date', 'time']] = df_merged['timestamp'].str.split('T', expand=True)
    
    df_merged = df_merged.drop(columns=['timestamp'], axis=1)
    
    return df_merged

In [9]:
# day_zero
df_final = convert_json_into_dataframe(pricesJ_day_zero, demandJ_day_zero, emissionsJ_day_zero)

# yesterday
df_yesterday = convert_json_into_dataframe(pricesJ_yesterday, demandJ_yesterday, emissionsJ_day_zero)
# Repetimos las emisiones del día cero
df_final = pd.concat([df_final, df_yesterday], ignore_index=True)


# today
df_today = convert_json_into_dataframe(pricesJ_today, demandJ_today, emissionsJ_day_zero, today=True)
df_final = pd.concat([df_final, df_today], ignore_index=True)


In [10]:
df_final.to_csv('data.csv', index=False)

In [11]:
del df_final

# TODO: CHANGE THIS READ_CSV

In [12]:
df = pd.read_csv('data.csv',
                 parse_dates={'data': ['date', 'time']},
                 infer_datetime_format=True,
                 dayfirst=False,
                 index_col='data'
                )


df.head()

,price,demand,emissions
data,,,
2022-02-11 00:00:00,254.30,26792,12287.250
2022-02-11 01:00:00,233.71,25817,11883.631
2022-02-11 02:00:00,232.08,24690,10940.722
2022-02-11 03:00:00,222.96,24232,11452.998
2022-02-11 04:00:00,219.90,23537,12749.593


# Preprocessing

In [13]:
# Normalization/Standarization params
train_mean = pd.Series(data = [0.025869, -0.074058, 0.272317], index = ['price', 'demand', 'emissions'])
train_std = pd.Series(data = [22.190049, 1295.266304, 537.640099], index = ['price', 'demand', 'emissions'])

## Differencing

In [14]:
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.Series(diff)

In [15]:
diff_price = difference(df['price'])
diff_emissions = difference(df['emissions'])
diff_demand = difference(df['demand'])

In [16]:
s1 = pd.Series([0.0])
diff_price = s1.append(diff_price, ignore_index=True)
diff_emissions = s1.append(diff_emissions, ignore_index=True)
diff_demand = s1.append(diff_demand, ignore_index=True)

In [17]:
df['price'] = diff_price.values
df['demand'] = diff_demand.values
df['emissions'] = diff_emissions.values

In [18]:
df.head()

,price,demand,emissions
data,,,
2022-02-11 00:00:00,0.00,0.0,0.000
2022-02-11 01:00:00,-20.59,-975.0,-403.619
2022-02-11 02:00:00,-1.63,-1127.0,-942.909
2022-02-11 03:00:00,-9.12,-458.0,512.276
2022-02-11 04:00:00,-3.06,-695.0,1296.595


## Normalization

In [19]:
df = (df-train_mean)/train_std

## Select subset with length=input_size

In [20]:
df = df[len(df)-48:]

# Convert the dataframe into a window (CHANGE: fill labels with zeros)

In [21]:
OUT_STEPS = 24
multi_window = WindowGenerator(input_width=48, label_width=OUT_STEPS, shift=OUT_STEPS, train_df=df, test_df=df)
multi_window

Total window size: 72
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
Label indices: [48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71]
Label column name(s): None